In [1]:
import cv2
import numpy as np

In [2]:
def canny_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges


In [3]:
def region_of_interest(image):
    height = image.shape[0]
    width = image.shape[1]
    polygons = np.array([[(50, height), (width-50, height), (width//2, height//3)]], dtype=np.int32)
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image


In [4]:
def draw_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)
    return line_image


In [5]:
def hough_lines_detection(image):
    lines = cv2.HoughLinesP(
        image,
        rho=1,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=100,
        maxLineGap=30
    )
    return lines


In [6]:
def filter_lines_by_slope(lines, min_slope=0.5, max_slope=2.0):
    filtered_lines = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else float('inf')
        if min_slope <= abs(slope) <= max_slope:
            filtered_lines.append(line)
    return filtered_lines


In [7]:
def estimate_speed(prev_position, curr_position, time_interval):
    displacement = np.linalg.norm(np.array(curr_position) - np.array(prev_position))
    speed = displacement / time_interval
    return speed


In [8]:
def lane_detection_pipeline(image, prev_position=None):
    canny_edges = canny_edge_detection(image)
    cropped_edges = region_of_interest(canny_edges)
    lines = hough_lines_detection(cropped_edges)
    
    if lines is not None and len(lines) > 0:
        lines = filter_lines_by_slope(lines, min_slope=0.2, max_slope=2.0)
    else:
        lines = []

    line_image = draw_lines(image, lines)
    combined_image = cv2.addWeighted(image, 0.8, line_image, 1.5, 0)
    
    if prev_position is not None and lines:
        curr_position = (lines[0][0][0], lines[0][0][1])
        speed = estimate_speed(prev_position, curr_position, time_interval=1)
        cv2.putText(combined_image, f"Speed: {speed:.2f} m/s", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return combined_image, (lines[0][0][0], lines[0][0][1]) if lines else prev_position


In [9]:
def real_time_lane_detection():
    cap = cv2.VideoCapture(0)
    prev_position = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame, prev_position = lane_detection_pipeline(frame, prev_position)
        
        cv2.imshow("Real-Time Lane Detection", processed_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
real_time_lane_detection()
